<img src="lalogonavy.png">

<h1>
<p style="text-align: center;">Configuring Your First IoT Device</p>
</h1>

<img src="first-iot-device-lab.png">

### Let's install some requirements and set some variables:

In [ ]:
!pip install mysql-connector boto3 cython PyHamcrest

### Uncomment the commented variables and add the values you obtain from the Learning Activity Credentials screen. Example values have been provided in order to help ensure you use the right ones. 

In [ ]:
#endpoint = "35.172.134.237" # This is the IP for the Database
#lambdaRole = "arn:aws:iam::410253224468:role/cfst-1048-95df4023b0fe7844f9265c3f01cb-ALambdaRole-1396SFBQTLRG1" # This is the role for the lambda function
username = "cloud_user"
password = "linuxacademy"
topic = '/stream/viewerdata/'
tablename = "Movies"

In [5]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import sys
import random
from time import sleep

### Let's stream our data:

In [20]:
import boto3
import random
# dynamodb = boto3.resource('dynamodb',  region_name='us-east-1')
# table = dynamodb.Table(tablename)

choices = ['yes', 'no']
i = 0
with open("moviedata.json") as json_file:

    movies = json.load(json_file, parse_float = decimal.Decimal)
    for movie in movies:
        i = i + 1
        customerID = random.randrange(1000,1100)
        year = int(movie['year'])
        title = movie['title']
        star = movie['actors'][0]
        rating = movie['rating']
        running_time = movie['running_time_secs']
        enjoyed = random.choice(choices)

        print("Customer", customerID, "watched movie:", year, title, star, rating, running_time, enjoyed)

#         table.put_item(
#            Item={
#                'customerID': customerID
#                'year': year,
#                'title': title,
#                'actor': star,
#                'rating': rating,
#                'running_time': running_time,
#                'enjoyed' : enjoyed
#             }
#         )
        sleep(2)


Customer 1049 watched movie: 2013 Rush Daniel Bruhl 8.3 7380 yes
Customer 1094 watched movie: 2013 Prisoners Hugh Jackman 8.2 9180 no
Customer 1014 watched movie: 2013 This Is the End James Franco 7.2 6420 yes
Customer 1010 watched movie: 2013 Insidious: Chapter 2 Patrick Wilson 7.1 6360 yes
Customer 1078 watched movie: 2013 World War Z Brad Pitt 7.1 6960 no
Customer 1055 watched movie: 2013 Now You See Me Jesse Eisenberg 7.3 6900 yes
Customer 1080 watched movie: 2013 Gravity Sandra Bullock 8.2 5400 yes
Customer 1010 watched movie: 2013 We're the Millers Jason Sudeikis 7.2 6600 yes
Customer 1099 watched movie: 2013 Riddick Vin Diesel 6.8 7140 yes
Customer 1092 watched movie: 2013 The Family Robert De Niro 6.5 6660 yes
Customer 1029 watched movie: 2013 Star Trek Into Darkness Chris Pine 7.9 7920 no
Customer 1059 watched movie: 2013 After Earth Jaden Smith 4.9 6000 yes
Customer 1069 watched movie: 2013 The Great Gatsby Leonardo DiCaprio 7.3 8580 yes
Customer 1094 watched movie: 2013 We A

Customer 1078 watched movie: 2013 The Heat Sandra Bullock 6.9 7020 yes
Customer 1067 watched movie: 2009 Inglourious Basterds Brad Pitt 8.3 9180 yes
Customer 1036 watched movie: 1972 The Godfather Marlon Brando 9.2 10500 yes
Customer 1077 watched movie: 2013 Filth James McAvoy 7.2 5820 no
Customer 1028 watched movie: 1999 Fight Club Brad Pitt 8.8 8340 no
Customer 1080 watched movie: 2013 Escape from Tomorrow Roy Abramsohn 5.9 5400 no
Customer 1010 watched movie: 1976 Carrie Sissy Spacek 7.4 5880 yes
Customer 1004 watched movie: 2013 Enough Said Julia Louis-Dreyfus 7.5 5580 yes
Customer 1025 watched movie: 2012 Prometheus Noomi Rapace 7.1 7440 yes
Customer 1080 watched movie: 2006 The Departed Leonardo DiCaprio 8.5 9060 yes
Customer 1023 watched movie: 1994 Dumb & Dumber Jim Carrey 7.2 6420 no
Customer 1084 watched movie: 2013 Romeo and Juliet Hailee Steinfeld 5 7080 no
Customer 1039 watched movie: 2009 Cloudy with a Chance of Meatballs Anna Faris 7 5400 yes
Customer 1086 watched movie:

Customer 1054 watched movie: 1999 The Matrix Keanu Reeves 8.7 8160 no
Customer 1088 watched movie: 2010 Iron Man 2 Robert Downey Jr. 7 7440 no
Customer 1059 watched movie: 1994 Forrest Gump Tom Hanks 8.7 8520 yes
Customer 1026 watched movie: 2013 Parker Jason Statham 6.1 7080 no
Customer 1096 watched movie: 2004 The Chronicles of Riddick Vin Diesel 6.5 7140 no
Customer 1050 watched movie: 1994 The Lion King Matthew Broderick 8.4 5340 no
Customer 1049 watched movie: 2012 Jack Reacher Tom Cruise 6.9 7800 yes
Customer 1058 watched movie: 2011 The Girl with the Dragon Tattoo Daniel Craig 7.8 9480 yes
Customer 1004 watched movie: 2009 Avatar Sam Worthington 7.9 9720 yes
Customer 1082 watched movie: 2012 End of Watch Jake Gyllenhaal 7.6 6540 no
Customer 1008 watched movie: 2012 Snow White and the Huntsman Kristen Stewart 6.1 7620 no
Customer 1016 watched movie: 2013 Mama Jessica Chastain 6.3 6000 no
Customer 1017 watched movie: 2012 This Is 40 Paul Rudd 6.2 8040 no
Customer 1085 watched movi

Customer 1043 watched movie: 2000 Snatch. Jason Statham 8.3 6240 no
Customer 1024 watched movie: 1996 Independence Day Will Smith 6.8 8700 yes
Customer 1086 watched movie: 2013 +1 Rhys Wakefield 5.6 5700 no
Customer 1032 watched movie: 1996 Romeo + Juliet Leonardo DiCaprio 6.8 7200 yes
Customer 1043 watched movie: 1975 The Rocky Horror Picture Show Tim Curry 7.2 6000 yes
Customer 1059 watched movie: 2013 Getaway Ethan Hawke 4.1 5400 no
Customer 1049 watched movie: 2005 Hooligans Elijah Wood 7.4 6540 no
Customer 1066 watched movie: 2012 Total Recall Colin Farrell 6.2 7080 yes
Customer 1005 watched movie: 2007 28 Weeks Later Jeremy Renner 7 6000 yes
Customer 1070 watched movie: 1992 Reservoir Dogs Harvey Keitel 8.4 5940 no
Customer 1091 watched movie: 1993 Dazed and Confused Jason London 7.6 6120 yes
Customer 1028 watched movie: 2002 Spider-Man Tobey Maguire 7.3 7260 no
Customer 1015 watched movie: 2006 The Covenant Steven Strait 4.9 5820 no
Customer 1052 watched movie: 2012 Battleship A

KeyboardInterrupt: 

### All Done! Awesome Job!